In [ ]:
'''
Feature Importance for Heart Disease Dataset 
'''

In [2]:
import pandas as pd

data = pd.read_csv("heart.csv")
df = pd.DataFrame(data)

df.head(10)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
5,39,M,NAP,120,339,0,Normal,170,N,0.0,Up,0
6,45,F,ATA,130,237,0,Normal,170,N,0.0,Up,0
7,54,M,ATA,110,208,0,Normal,142,N,0.0,Up,0
8,37,M,ASY,140,207,0,Normal,130,Y,1.5,Flat,1
9,48,F,ATA,120,284,0,Normal,120,N,0.0,Up,0


In [5]:

categorical_columns = ['ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']

#ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), categorical_columns)], remainder='passthrough')
#df_encoded = ct.fit_transform(df)

#column_names = ct.get_feature_names_out()
#df_encoded = pd.DataFrame(df_encoded, columns=column_names)

# # Display the first 10 rows of the transformed DataFrame
# print(df_encoded.head(10))

#df_encoded = pd.get_dummies(df, columns=categorical_columns, drop_first=True)
#df_encoded.head(10)

df.loc[df.Sex == 'M', 'Sex'] = 1
df.loc[df.Sex == 'F', 'Sex'] = 0

for feature in categorical_columns:
    onehot = pd.get_dummies(df[feature], prefix=feature)
    df = df.drop(feature, axis=1)
    df = df.join(onehot)

df.head(10)
     

,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ExerciseAngina_N,ExerciseAngina_Y,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
0,40,1,140,289,0,172,0.0,0,0,1,0,0,0,1,0,1,0,0,0,1
1,49,0,160,180,0,156,1.0,1,0,0,1,0,0,1,0,1,0,0,1,0
2,37,1,130,283,0,98,0.0,0,0,1,0,0,0,0,1,1,0,0,0,1
3,48,0,138,214,0,108,1.5,1,1,0,0,0,0,1,0,0,1,0,1,0
4,54,1,150,195,0,122,0.0,0,0,0,1,0,0,1,0,1,0,0,0,1
5,39,1,120,339,0,170,0.0,0,0,0,1,0,0,1,0,1,0,0,0,1
6,45,0,130,237,0,170,0.0,0,0,1,0,0,0,1,0,1,0,0,0,1
7,54,1,110,208,0,142,0.0,0,0,1,0,0,0,1,0,1,0,0,0,1
8,37,1,140,207,0,130,1.5,1,1,0,0,0,0,1,0,0,1,0,1,0
9,48,0,120,284,0,120,0.0,0,0,1,0,0,0,1,0,1,0,0,0,1


In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

scaler = StandardScaler()

y = df['HeartDisease'] 
x = df.drop(columns='HeartDisease') 

x_scaled = scaler.fit_transform(x) 

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state = 1)

In [20]:
'''Ridge Regression'''
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn import metrics

possible_alphas = [0.001, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.0001, 0.00001, 0.003, 1]
for i in range(2, 34):
    possible_alphas.append(i) 

columns = x_train.columns.tolist() 
ridge_model = RidgeCV(alphas = possible_alphas, scoring='r2')
ridge_model = ridge_model.fit(x_train, y_train)
y_pred_ridge = ridge_model.predict(x_test)
y_train_pred_ridge = ridge_model.predict(x_train)
ridge_test_error = metrics.mean_squared_error(y_test, y_pred_ridge)
ridge_train_error = metrics.mean_squared_error(y_train, y_train_pred_ridge) 

print("Ridge Mean Squared error: ", ridge_test_error)
print("Ridge Training Mean Squared Error: ", ridge_train_error)
print('alpha: ', ridge_model.alpha_)

ridge_coefficients_list = ridge_model.coef_.tolist()
ridge_coefficients = {} 
for i in range(0, len(ridge_coefficients_list)):
    ridge_coefficients[ridge_coefficients_list[i]] = columns[i]

ridge_coefficients_list.sort(reverse=True) 
sorted_dictionary = {}
keys = list(ridge_coefficients.keys())
print(keys)
for i in range(0, len(ridge_coefficients_list)): 
    ind = keys.index(ridge_coefficients_list[i]) 
    sorted_dictionary[keys[ind]] = ridge_coefficients[keys[ind]] 

print('Sorted coefficients: ', sorted_dictionary) 



Ridge Mean Squared error:  0.08673086808773343
Ridge Training Mean Squared Error:  0.11088044544155308
alpha:  19.0
[0.002492232564280439, 0.14363090354024566, 0.0008228125464300184, -0.0006426995216166631, 0.09590456539578762, -0.0007291014666641615, 0.05366218162377798, 0.15627912256783155, -0.10052329011565642, -0.07190595738345451, 0.016150124931279316, 0.00155729457183644, -0.0011057691331516906, -0.00045152543868465234, -0.0720860917859916, 0.07208609178599165, -0.009162173771210112, 0.16526056009095452, -0.15609838631974443]
Sorted coefficients:  {0.16526056009095452: 'ST_Slope_Flat', 0.15627912256783155: 'ChestPainType_ASY', 0.14363090354024566: 'Sex', 0.09590456539578762: 'FastingBS', 0.07208609178599165: 'ExerciseAngina_Y', 0.05366218162377798: 'Oldpeak', 0.016150124931279316: 'ChestPainType_TA', 0.002492232564280439: 'Age', 0.00155729457183644: 'RestingECG_LVH', 0.0008228125464300184: 'RestingBP', -0.00045152543868465234: 'RestingECG_ST', -0.0006426995216166631: 'Cholesterol

In [22]:
'''Lasso Regression'''
lasso_model = LassoCV(alphas = [1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, .1, 0.09, 0.08, 0.07, 0.01, 0.001, 0.0001, 0.0002, 0.0003, 0.0004, 0.00001], max_iter = 10000000)
lasso_model = lasso_model.fit(x_train, y_train)
y_pred_lasso = lasso_model.predict(x_test)
y_pred_lasso_train = lasso_model.predict(x_train)
lasso_test_error = metrics.mean_squared_error(y_test, y_pred_lasso)
lasso_train_error = metrics.mean_squared_error(y_train, y_pred_lasso_train) 
print("Lasso Mean Squared Error: ", lasso_test_error)
print("Lasso Train Mean Squared Error: ", lasso_train_error)
print("Lasso Alpha: ", lasso_model.alpha_)

lasso_coefficients_list = lasso_model.coef_.tolist()
lasso_coefficients = {} 
for i in range(0, len(lasso_coefficients_list)):
    lasso_coefficients[lasso_coefficients_list[i]] = columns[i]

lasso_coefficients_list.sort(reverse=True) 
sorted_dictionary = {}
keys = list(lasso_coefficients.keys())

for i in range(0, len(lasso_coefficients_list)): 
    ind = keys.index(lasso_coefficients_list[i]) 
    sorted_dictionary[keys[ind]] = lasso_coefficients[keys[ind]] 

print('Sorted coefficients: ', sorted_dictionary) 


Lasso Mean Squared Error:  0.08564464937668234
Lasso Train Mean Squared Error:  0.1106808981348282
Lasso Alpha:  0.001
Sorted coefficients:  {0.2287308630657632: 'ChestPainType_ASY', 0.18267713782873662: 'ST_Slope_Flat', 0.15909513876303774: 'Sex', 0.10259481848930137: 'FastingBS', 0.0692022149415966: 'ChestPainType_TA', 0.05108502198685188: 'Oldpeak', 0.0024584939471784966: 'Age', 0.0008114502242932477: 'RestingBP', 2.531543026016977e-18: 'ExerciseAngina_Y', 0.0: 'ST_Slope_Down', -0.0005539552816242006: 'MaxHR', -0.0006188972400374164: 'Cholesterol', -0.008907319475829704: 'ChestPainType_NAP', -0.04055939130107977: 'ChestPainType_ATA', -0.1396204862696593: 'ExerciseAngina_N', -0.15398286452046475: 'ST_Slope_Up'}
